In [ ]:
!pip install google-api-python-client youtube-transcript-api pandas openpyxl

##  Import Required Libraries

In [4]:
# Import necessary libraries
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound
import pandas as pd
import time
import random
import sys
import os

# Data collection from YoutubeAPI

In [5]:
# Add the parent directory (where config.py resides) to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

try:
    from config import API_KEY
except ImportError as e:
    raise RuntimeError(f"API key not found! Ensure 'config.py' exists in the parent directory. Error: {e}")

# Set up the YouTube Data API client
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Function to fetch video categories
def get_video_categories(region_code="US"):
    categories = {}  # Declare categories as a local dictionary to return
    try:
        request = youtube.videoCategories().list(
            part="snippet",
            regionCode=region_code
        )
        response = request.execute()
        for item in response.get("items", []):
            categories[item["id"]] = item["snippet"]["title"]
    except Exception as e:
        print(f"Error fetching video categories: {e}")
    return categories

# Function to fetch video IDs for a specific category with children-friendly content
def fetch_video_ids_for_kids_category(category_id, max_results=10):
    video_ids = []
    try:
        # Fetch videos with strict safe search enabled for children-friendly content
        request = youtube.search().list(
            part="id",
            type="video",
            videoCategoryId=category_id,
            maxResults=max_results,
            # safeSearch='strict'  # Ensures children-friendly videos
        )
        response = request.execute()
        for item in response.get("items", []):
            video_ids.append(item["id"]["videoId"])
        
        # Shuffle to ensure randomness if required
        random.shuffle(video_ids)
        video_ids = video_ids[:max_results]  # Restrict to the desired number of results
    except Exception as e:
        print(f"Error fetching video IDs for category {category_id}: {e}")
    return video_ids

# Function to fetch video IDs for a specific category
def fetch_video_ids_for_category(category_id, max_results=10):
    video_ids = []
    try:
        request = youtube.search().list(
            part="id",
            type="video",
            videoCategoryId=category_id,
            maxResults=50  # Fetch more than 10 to enable random selection
        )
        response = request.execute()
        for item in response.get("items", []):
            video_ids.append(item["id"]["videoId"])
        
        # Randomly shuffle the video IDs and pick the first `max_results` video IDs
        random.shuffle(video_ids)
        video_ids = video_ids[:max_results]
    except Exception as e:
        print(f"Error fetching video IDs for category {category_id}: {e}")
    return video_ids

# Function to fetch video metadata
def get_video_metadata(video_ids):
    metadata = []
    try:
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=",".join(video_ids)
        )
        response = request.execute()
        for video in response.get("items", []):
            snippet = video.get("snippet", {})
            content_details = video.get("contentDetails", {})
            statistics = video.get("statistics", {})
            
            metadata.append({
                "video_id": video.get("id"),
                "title": snippet.get("title", ""),
                "description": snippet.get("description", ""),
                "published_at": snippet.get("publishedAt", ""),
                "channel_title": snippet.get("channelTitle", ""),
                "tags": ", ".join(snippet.get("tags", [])) if snippet.get("tags") else "",
                "category_id": snippet.get("categoryId", ""),
                "duration": content_details.get("duration", ""),
                "view_count": statistics.get("viewCount", 0),
                "like_count": statistics.get("likeCount", 0),
                "dislike_count": statistics.get("dislikeCount", 0),
                "default_audio_language": snippet.get("defaultAudioLanguage", ""),
            })
    except Exception as e:
        print(f"Error fetching metadata: {e}")
    return metadata

# Function to fetch video transcript
def get_video_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        transcript_text = " ".join([t['text'] for t in transcript])
        return transcript_text
    except NoTranscriptFound:
        return ""
    except Exception as e:
        print(f"Error fetching transcript for {video_id}: {e}")
        return ""

# Step 2: Collect 10 random videos per category
VIDEOS_PER_CATEGORY = 5
BATCH_SIZE = 10  # Define batch size for processing metadata
all_data = []

## Fetch all category without kids safe

In [6]:
# Step 1: Fetch all YouTube categories
categories = get_video_categories()
print(f"Found {len(categories)} categories: {categories}")

for category_id, category_name in categories.items():
    print(f"Collecting data for category: {category_name} (ID: {category_id})")
    video_ids = fetch_video_ids_for_category(category_id, max_results=VIDEOS_PER_CATEGORY)
    video_ids = fetch_video_ids_for_kids_category(category_id, max_results=VIDEOS_PER_CATEGORY)
    print(f"Collected {len(video_ids)} video IDs for category {category_name}.")
    
    # Fetch metadata and transcripts in batches
    for i in range(0, len(video_ids), BATCH_SIZE):
        batch_ids = video_ids[i:i+BATCH_SIZE]
        
        # Fetch metadata
        metadata = get_video_metadata(batch_ids)
        
        # Fetch transcripts
        for item in metadata:
            video_id = item["video_id"]
            transcript = get_video_transcript(video_id)
            item["transcript"] = transcript
        
        all_data.extend(metadata)
        print(f"Processed {i + len(batch_ids)} / {len(video_ids)} videos for category {category_name}.")
        time.sleep(random.uniform(1, 3))  # Pause to avoid rate limits

Found 32 categories: {'1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music', '15': 'Pets & Animals', '17': 'Sports', '18': 'Short Movies', '19': 'Travel & Events', '20': 'Gaming', '21': 'Videoblogging', '22': 'People & Blogs', '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', '28': 'Science & Technology', '29': 'Nonprofits & Activism', '30': 'Movies', '31': 'Anime/Animation', '32': 'Action/Adventure', '33': 'Classics', '34': 'Comedy', '35': 'Documentary', '36': 'Drama', '37': 'Family', '38': 'Foreign', '39': 'Horror', '40': 'Sci-Fi/Fantasy', '41': 'Thriller', '42': 'Shorts', '43': 'Shows', '44': 'Trailers'}
Collected 5 video IDs for category Film & Animation.
Error fetching transcript for lTjkFcVlypQ: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=lTjkFcVlypQ! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible 

## Fetch only kids category video with safeSearch attribute

In [6]:
# Fetch only kids safe content:
category_id = 24
video_ids = fetch_video_ids_for_kids_category(category_id, max_results=VIDEOS_PER_CATEGORY)
# print(f"Collected {len(video_ids)} video IDs for category {category_name}.")

# Fetch metadata and transcripts in batches
for i in range(0, len(video_ids), BATCH_SIZE):
    batch_ids = video_ids[i:i+BATCH_SIZE]
    
    # Fetch metadata
    metadata = get_video_metadata(batch_ids)
    
    # Fetch transcripts
    for item in metadata:
        video_id = item["video_id"]
        transcript = get_video_transcript(video_id)
        item["transcript"] = transcript
    
    all_data.extend(metadata)
    # print(f"Processed {i + len(batch_ids)} / {len(video_ids)} videos for category {category_name}.")
    time.sleep(random.uniform(1, 3))  # Pause to avoid rate limits

all_data

Error fetching transcript for FYiQmFGRHd8: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=FYiQmFGRHd8! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
Error fetching transcript for efp3sv7t1RQ: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=efp3sv7t1RQ! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtu

[{'video_id': 'OGUDOc-EvGI',
  'title': "WE'RE MAKING A MANGA",
  'description': "Manga Contest Details: https://wordlessmanga.com/eng.html\nManga Contest FAQ: https://x.com/KADOKAWAdigipub/status/1866081205290205245\nJoin our Patreon! https://www.patreon.com/spilledinkling\nDaidus'  main channel:  @Daidus  \nDaidus' Twitter: https://twitter.com/Daidus_OTL\nDaidus' Instagram: https://www.instagram.com/daidus_art/?hl=en\nEmirichu's main channel:  @Emirichu  \nEmily's Twitter: https://twitter.com/EmirichuYT\nEmily's Instagram: https://www.instagram.com/emirichuu/?hl=en\n\nSPILLED INK! Instagram: https://www.instagram.com/spilledinkling?igsh=MTdyb2Y0cHBsNm5rNg%3D%3D&utm_source=qr\n\nEdited by the Mudan Media Group @marukudeibu \n\nEpisode 32",
  'published_at': '2024-12-09T14:56:27Z',
  'channel_title': 'Spilled Ink',
  'tags': 'manga, manga contest, manga competition, drawing, drawing manga, drawing anime, anime, original manga, tradtional manga, rough draft, name, art, manga editor, kad

## Save the dataframe

In [8]:
all_data

[{'video_id': 'lTjkFcVlypQ',
  'title': '🔴 LIVE! Phineas and Ferb Season 1 Full Episodes! | @disneyxd',
  'description': 'Ferb! I know what we\'re gonna do today! Watch the full first season of Phineas and Ferb! While you\'re watching all the episodes from season 1, find Perry the Platypus!\n\nSeason 1 Episode 1 Rollercoaster / Candace Loses Her Head\nPhineas and Ferb build the coolest rollercoaster ever in their backyard! On Candace\'s birthday, the family takes a trip to Mount Rushmore.\n\nSeason 1 Episode 2 The Fast and the Phineas / Lawn Gnome Beach Party of Terror!\nThe boys convert Mom\'s car into a giant remote-controlled car and enter it in a race! The boys decide to make a tropical beach paradise in their very own backyard.\n\nSeason 1 Episode 3 The Magnificent Few / S\'winter\nPhineas and Ferb become real cowboys as they wrangle a stampede of cattle through Danville\'s city streets. It\'s wintertime in the summertime!\n\nSeason 1 Episode 4 Are You My Mummy? / Flop Starz\nPhin

In [9]:
# Convert data to a DataFrame
df = pd.DataFrame(all_data)

# Define the output file path
output_file = "../data/raw/inference_sample_1.xlsx"

# Create the directory if it does not exist
output_dir = os.path.dirname(output_file)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the collected data to an Excel file
df.to_excel(output_file, index=False, engine='openpyxl')
print(f"Data saved to {output_file}")

Data saved to ../data/raw/inference_sample_1.xlsx


## add a column "kids safe"

In [4]:
# Step: Read the Excel file into a DataFrame
df_output = pd.read_excel(output_file, engine='openpyxl')

# Step: Add a new column 'kids_safe' with value 1 for all rows
df_output['kids_safe'] = 1

# Print the DataFrame to confirm changes
print(df_output.head())

# Optionally, save the updated DataFrame back to the file
df_output.to_excel(output_file, index=False, engine='openpyxl')
print(f"Updated DataFrame with 'kids_safe' column saved to {output_file}")

      video_id                                              title  \
0  -aqYc5WPd-s  Dalek appeared to help the Allies in World War...   
1  IJbbpAhjboI  🔴 LIVE: Tom and Jerry | 1 Hour Compilation | T...   
2  InFOaemJcdU  Lechia Gdańsk - Pogoń Szczecin 1:1 | SKRÓT | E...   
3  Kq10UVgHeHQ        A LESSON OF MAGIC 🪄✨ | #anime #animemoments   
4  XviHNdfuB9Y  🎥 THREE BILLBOARDS OUTSIDE EBBONG, MISSOURI (2...   

                                         description          published_at  \
0                                                NaN  2024-12-09T03:43:41Z   
1  🚩 Watch more Tom and Jerry - https://goo.gl/RU...  2024-07-10T08:28:07Z   
2  Lechia Gdańsk 1:1 Pogoń Szczecin\n1:0 - Adam B...  2016-07-23T08:17:07Z   
3                                                NaN  2024-12-07T23:45:00Z   
4  A mother personally challenges the local autho...  2018-02-19T14:00:02Z   

          channel_title                                               tags  \
0         Brought Kooky               

## Combine metadata excel sheets.

In [7]:
import pandas as pd

# Load the Excel sheets into dataframes
df1 = pd.read_excel("/Users/shusritavenugopal/Documents/MSIS/FALL2024/InformationStorageAndRetrieval/video_classification_project/data/raw/video_metadata_by_category.xlsx")
df2 = pd.read_excel("/Users/shusritavenugopal/Documents/MSIS/FALL2024/InformationStorageAndRetrieval/video_classification_project/data/raw/video_metadata.xlsx")

# Verify dataframes are loaded correctly
print("DataFrame 1 (Sheet 1):")
df1.head()

DataFrame 1 (Sheet 1):


,video_id,title,description,published_at,channel_title,tags,category_id,duration,view_count,like_count,dislike_count,default_audio_language,transcript
0,crxHAtFqU4o,When a Pilot Sank a Submarine to Avenge a Friend,Prepare to embark on a voyage like no other as...,2024-11-28T23:15:08Z,Yarnhub,NaN,1,PT12M17S,505645,16878,0,en,"It's December 7 , 1941. In the Pacific, Lieute..."
1,qHecSpzXs3s,Fiona gave her best time to her younger siblin...,NaN,2024-11-28T15:48:00Z,Brianna Movies,NaN,1,PT1M,369560,27635,0,en-US,Miss Gallagher you want to tell me why your fa...
2,XviHNdfuB9Y,"🎥 THREE BILLBOARDS OUTSIDE EBBONG, MISSOURI (2...",A mother personally challenges the local autho...,2018-02-19T14:00:02Z,TRAILER CITY,"trailer, full movie trailer, new trailer, trai...",1,PT1M29S,82425,224,0,en,anyway trying to make me believe in reincarnat...
3,ObGVA1WOqyE,🎥 THE PRESTIGE (2006) | Full Movie Trailer in ...,After a tragic accident two stage magicians en...,2018-02-01T14:00:03Z,MOVIE PREDICTOR,"trailer, full movie trailer, official trailer,...",1,PT2M48S,444240,3246,0,en,surely come on no I can't hey how do you do it...
4,lTjkFcVlypQ,🔴 LIVE! Phineas and Ferb Season 1 Full Episode...,Ferb! I know what we're gonna do today! Watch ...,2024-10-18T18:01:23Z,Disney XD,"phineas and ferb, phineas and ferb full episod...",1,P0D,2480550,10100,0,en,NaN


In [9]:
df1.columns

Index(['video_id', 'title', 'description', 'published_at', 'channel_title',
       'tags', 'category_id', 'duration', 'view_count', 'like_count',
       'dislike_count', 'default_audio_language', 'transcript'],
      dtype='object')

In [10]:
df2.columns

Index(['Video ID', 'Title', 'Description', 'Published At', 'Channel Title',
       'Tags', 'Category ID', 'Category Label', 'Default Audio Language',
       'Transcript'],
      dtype='object')

In [11]:
# Standardize column names in dataset2 to match dataset1 for merging
df2 = df2.rename(columns={
    'Video ID': 'video_id',
    'Title': 'title',
    'Description': 'description',
    'Published At': 'published_at',
    'Channel Title': 'channel_title',
    'Tags': 'tags',
    'Category ID': 'category_id',
    'Category Label': 'Category Label',
    'Default Audio Language': 'default_audio_language',
    'Transcript': 'transcript'
})

# Add missing columns in df2
df2['duration'] = None
df2['view_count'] = None
df2['like_count'] = None
df2['dislike_count'] = None

# Combine the datasets
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined dataset to an Excel file
output_file = 'combined_dataset.xlsx'
combined_df.to_excel(output_file, index=False, engine='openpyxl')

print(f"Combined dataset saved to {output_file}")

Combined dataset saved to combined_dataset.xlsx


In [5]:
# File path of the second Excel file
output_file = "../data/raw/video_metadata_by_category_kids.xlsx"
second_file = "../data/raw/combined_dataset.xlsx"

# Step: Read both Excel files into DataFrames
df_output = pd.read_excel(output_file, engine='openpyxl')  # Our output file
df_second = pd.read_excel(second_file, engine='openpyxl')  # Existing file

# Step: Ensure both DataFrames have the same structure
# Add a 'kids_safe' column to the second DataFrame, leaving it empty
df_second['kids_safe'] = None

# Step: Merge the two DataFrames
merged_df = pd.concat([df_output, df_second], ignore_index=True)

# Print the first few rows of the merged DataFrame to verify
print(merged_df.head())

# Step: Save the merged DataFrame back to a new Excel file
merged_file = "../data/raw/master_dataset.xlsx"
merged_df.to_excel(merged_file, index=False, engine='openpyxl')
print(f"Merged data saved to {merged_file}")


      video_id                                              title  \
0  -aqYc5WPd-s  Dalek appeared to help the Allies in World War...   
1  IJbbpAhjboI  🔴 LIVE: Tom and Jerry | 1 Hour Compilation | T...   
2  InFOaemJcdU  Lechia Gdańsk - Pogoń Szczecin 1:1 | SKRÓT | E...   
3  Kq10UVgHeHQ        A LESSON OF MAGIC 🪄✨ | #anime #animemoments   
4  XviHNdfuB9Y  🎥 THREE BILLBOARDS OUTSIDE EBBONG, MISSOURI (2...   

                                         description          published_at  \
0                                                NaN  2024-12-09T03:43:41Z   
1  🚩 Watch more Tom and Jerry - https://goo.gl/RU...  2024-07-10T08:28:07Z   
2  Lechia Gdańsk 1:1 Pogoń Szczecin\n1:0 - Adam B...  2016-07-23T08:17:07Z   
3                                                NaN  2024-12-07T23:45:00Z   
4  A mother personally challenges the local autho...  2018-02-19T14:00:02Z   

          channel_title                                               tags  \
0         Brought Kooky               